# Example of using LiteQTL

This notebook showcase how to use LiteQTL to perform a QTL scan.

### Dataset: 
We use the BXD Genotype Database (GN Accession: GN600) and a sets of transcriptome data, UTHSCAffy MoGene 1.0 ST Spleen (GN Accession: GN283) from GeneNetwork. 

The genotype file includes 7321 markers by 198 BXD strains; 

The spleen dataset has data for 79 BXD strains and for 35556 transcripts. 

Data cleaning and wrangling was performed using R/qtl (Broman et al. 2003) and R/qtl2 (Broman et al. 2019). 

For details regarding data cleaning, please see [this script](https://github.com/senresearch/LiteQTL-G3-supplement/blob/master/code/clean-data.R)

In [6]:
# Loading packages
using LiteQTL
using DelimitedFiles
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


In [3]:
# Location of genotype file and phenotype file
geno_file = joinpath(@__DIR__, "..", "data", "processed", "spleen-bxd-genoprob.csv")
pheno_file = joinpath(@__DIR__, "..", "data","processed", "spleen-pheno-nomissing.csv")

# Whether to output whole LOD matrix or only maximum LOD of each phenptype? Default is false (only maximum)
export_matrix = false

# Location of output 
output_file = "output.csv"

# Setting BLAS threads as 16 threads. 
LiteQTL.set_blas_threads(16);

# Read in data.
G = LiteQTL.get_geno_data(geno_file, Float64)
Y = LiteQTL.get_pheno_data(pheno_file, Float64, transposed=false)
# getting geno and pheno file size.
n = size(Y,1) # number of individuals
m = size(Y,2) # number of Traits
p = size(G,2) # number of Markers
println("******* Indivuduals n: $n, Traits m: $m, Markers p: $p ****************");

Number of threads using: 16


┌ Info: Removing sex column of phenotype. 
└ @ LiteQTL /Users/chelseatrotter/.julia/dev/LiteQTL/src/data_io.jl:23


******* Indivuduals n: 79, Traits m: 35554, Markers p: 7321 ****************


In [ ]:
# run analysis. 
lod = LiteQTL.cpurun(Y, G,n,export_matrix);

@benchmark LiteQTL.cpurun(Y, G,n,export_matrix)

# Uncomment the following lines if want to run the same function but with GPU: 
# lod = LiteQTL.gpurun(Y, G,n,m,p)
# @benchmark LiteQTL.gpurun(Y, G,n,m,p)

In [ ]:
# write output to file
writedlm(joinpath(Base.@__DIR__, "..", "data", "results", output_file), lod, ',')